In [1]:
import numpy as np

In [2]:
def read_training_data(fname, D=None):
    """Given a file in appropriate format, and given a set D of features,
    returns the pair (A, b) consisting of
    a P-by-D matrix A and a P-vector b,
    where P is a set of patient identification integers (IDs).

    For each patient ID p,
      - row p of A is the D-vector describing patient p's tissue sample,
      - entry p of b is +1 if patient p's tissue is malignant, and -1 if it is benign.

    The set D of features must be a subset of the features in the data (see text).
    """
    file = open(fname)
    params = ["radius", "texture", "perimeter","area","smoothness","compactness","concavity","concave points","symmetry","fractal dimension"];
    stats = ["(mean)", "(stderr)", "(worst)"]
    feature_labels = set([y+x for x in stats for y in params])
    feature_map = {params[i]+stats[j]:j*len(params)+i for i in range(len(params)) for j in range(len(stats))}
    
    patient_ids = []
    feature_vectors = []
    patient_diagnoses = []
    for line in file:
        row = line.split(",")
        patient_ID = int(row[0])
        patient_ids.append(patient_ID)
        
        patient_diagnoses.append(-1 if row[1]=='B' else +1)
        feature_vectors.append([float(row[feature_map[key]+2]) for key in feature_labels])
    return feature_labels, patient_ids, np.matrix(feature_vectors), np.array(patient_diagnoses) 

In [3]:
labels, ids, matrix, vectors = read_training_data('train.data')

In [4]:
print(labels)

{'smoothness(stderr)', 'texture(mean)', 'concavity(worst)', 'texture(worst)', 'concave points(stderr)', 'area(worst)', 'perimeter(mean)', 'compactness(worst)', 'symmetry(stderr)', 'perimeter(stderr)', 'fractal dimension(stderr)', 'area(stderr)', 'area(mean)', 'fractal dimension(worst)', 'radius(stderr)', 'concave points(worst)', 'fractal dimension(mean)', 'smoothness(worst)', 'texture(stderr)', 'symmetry(worst)', 'radius(mean)', 'concavity(mean)', 'perimeter(worst)', 'radius(worst)', 'compactness(stderr)', 'smoothness(mean)', 'concave points(mean)', 'concavity(stderr)', 'symmetry(mean)', 'compactness(mean)'}


In [5]:
def signum(u):
    return np.array([1 if v >= 0 else -1 for k, v in np.ndenumerate(u)])

In [6]:
signum(np.array([10, -20, 5, -3, 6]))

array([ 1, -1,  1, -1,  1])

In [7]:
def fraction_wrong(A, b, w):
    hypers_size = len(b)
#     hypers = []
#     for row in A:
#         row = np.array(row)
#         hypers.append(row.dot(w)[0])
#     hypers = np.array(hypers)
#     diff = signum(hypers).dot(b)

#     print(hypers_size)
#     print(signum(A.dot(w)).dot(b))
    hypers = signum(A.dot(w))
    diff = hypers.dot(b)
    diff = (hypers_size - diff) * 0.5
    ratio  = diff / hypers_size
    
    return ratio
    

In [8]:
w = [v for v in np.random.rand(30, 1)]
print(w)
# w = [1 for v in range(30)]
fraction_wrong(matrix, vectors, w)

[array([ 0.54425458]), array([ 0.12380031]), array([ 0.72951551]), array([ 0.85752237]), array([ 0.33779874]), array([ 0.76311652]), array([ 0.95045641]), array([ 0.92689646]), array([ 0.20778802]), array([ 0.60370603]), array([ 0.05121226]), array([ 0.80982389]), array([ 0.39198015]), array([ 0.99373178]), array([ 0.42771841]), array([ 0.26688645]), array([ 0.18703499]), array([ 0.21834495]), array([ 0.70857123]), array([ 0.89021576]), array([ 0.89174386]), array([ 0.49977356]), array([ 0.19065938]), array([ 0.9954612]), array([ 0.16659588]), array([ 0.08776235]), array([ 0.62683709]), array([ 0.94786931]), array([ 0.66234334]), array([ 0.39400408])]


0.51333333333333331

In [9]:
def loss(A, b, w):
    hypersis = np.squeeze(np.asarray(A.dot(w)))
    error = hypersis - b
    return error.dot(error)

In [10]:
w = [v for v in np.random.rand(30, 1)]
loss(matrix, vectors, w)

314937680.05090284

In [11]:
w = [v for v in np.random.rand(30, 1)]
loss(matrix, vectors, w)

534139284.63835913

In [12]:
w = [v for v in np.random.rand(30, 1)]
loss(matrix, vectors, w)

22226897.337725833

In [13]:
def find_grad(A, b, w):
#     hypersis = np.squeeze(np.asarray(A.dot(w)))
#     error = np.transpose(hypersis - b)
#     grad = np.squeeze(np.asarray(error.dot(A)))
#     return grad * 2
    
    hypersis = np.squeeze(np.squeeze(np.asarray(A.dot(w))).dot(A))
    ba = b.dot(A)
    error = np.transpose(hypersis - ba)
    return np.squeeze(np.asarray(error)) * 2

In [14]:
w = [v for v in np.random.rand(30, 1)]
find_grad(matrix, vectors, w)

array([  3.43177423e+03,   1.01633085e+07,   1.79108275e+05,
         1.36562641e+07,   6.73047702e+03,   6.19196282e+08,
         5.33147516e+07,   1.56797504e+05,   1.03966894e+04,
         1.93837043e+06,   1.97887635e+03,   3.16201858e+07,
         4.31944893e+08,   4.32998343e+04,   2.73530146e+05,
         7.50193606e+04,   3.10543410e+04,   6.88442994e+04,
         6.01631784e+05,   1.53518409e+05,   8.11870735e+06,
         6.38305535e+04,   6.43290352e+07,   9.68770630e+06,
         1.47156042e+04,   4.96226587e+04,   3.53835449e+04,
         1.85895785e+04,   9.38977024e+04,   6.29838466e+04])

In [15]:
def gradient_descent_step (A, b, w, sigma):
    return w - find_grad(A, b, w) * sigma

In [16]:
def gradient_descent (A, b, w, sigma, T):
    for t in range(T):
        w = gradient_descent_step(A, b, w, sigma)
#         print(loss(A, b, w))
    return w

In [17]:
w = [1 for v in range(30)]
w = gradient_descent(matrix, vectors, w, 2 * 10 ** -9, 30)
print(loss(matrix, vectors, w))

3.18014007384e+13


In [18]:
w = [1 for v in range(30)]
w = gradient_descent(matrix, vectors, w, 10 ** -9, 30)
print(loss(matrix, vectors, w))

1867476.16499


In [19]:
w = [0 for v in range(30)]
w = gradient_descent(matrix, vectors, w, 10 ** -9, 30)
print(loss(matrix, vectors, w))

251.010760796


In [20]:
w = [0 for v in range(30)]
w = gradient_descent(matrix, vectors, w, 10 ** -9, 60)
print(loss(matrix, vectors, w))

239.281498848


In [21]:
import random

w = [random.random() for v in range(30)]
w = gradient_descent(matrix, vectors, w, 10 ** -9, 30)
print(loss(matrix, vectors, w))

249556.293138
